In [ ]:
!pip install --upgrade tensorflow
!pip list
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 30.2 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [ ]:
import wikipedia
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Specify the exact page titles for each topic
topics = ["Artificial intelligence", "Neural Networks", "Deep Learning", "Artificial General Intelligence", "Robotics"]

combined_content = ""

for topic in topics:
    try:
        # Get the page for the topic
        page = wikipedia.page(topic)

        # Add the full content of the page to the combined content
        combined_content += page.content + "\n"
    except wikipedia.exceptions.PageError:
        print(f"The page for {topic} does not exist.")
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"The page for {topic} is a disambiguation page. Possible pages include:\n{e.options}")

# Create a character-level tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(combined_content)

# Convert text to sequences
sequences = tokenizer.texts_to_sequences([combined_content])[0]

# Create input sequences
input_sequences = []
for i in range(1, len(sequences)):
    n_gram_sequence = sequences[max(0, i - 100):i+1]
    input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

# Define the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 64, input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)




/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
8934/8934 ━━━━━━━━━━━━━━━━━━━━ 71s 7ms/step - accuracy: 0.3150 - loss: 2.4135
Epoch 2/100
8934/8934 ━━━━━━━━━━━━━━━━━━━━ 83s 8ms/step - accuracy: 0.4938 - loss: 1.7483
Epoch 3/100
8934/8934 ━━━━━━━━━━━━━━━━━━━━ 80s 8ms/step - accuracy: 0.5415 - loss: 1.5698
Epoch 4/100
8934/8934 ━━━━━━━━━━━━━━━━━━━━ 83s 8ms/step - accuracy: 0.5663 - loss: 1.4754
Epoch 5/100
8934/8934 ━━━━━━━━━━━━━━━━━━━━ 80s 8ms/step - accuracy: 0.5812 - loss: 1.4179
Epoch 6/100
8934/8934 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - accuracy: 0.5893 - loss: 1.3815
Epoch 7/100
8934/8934 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - accuracy: 0.5982 - loss: 1.3503
Epoch 8/100
8934/8934 ━━━━━━━━━━━━━━━━━━━━ 67s 8ms/step - accuracy: 0.6025 - loss: 1.3284
Epoch 9/100
8934/8934 ━━━━━━━━━━━━━━━━━━━━ 84s 8ms/step - accuracy: 0.6075 - loss: 1.3082
Epoch 10/100
8934/8934 ━━━━━━━━━━━━━━━━━━━━ 81s 8ms/step - accuracy: 0.6119 - loss: 1.2898
Epoch 11/100
8934/8934 ━━━━━━━━━━━━━━━━━━━━ 83s 8ms/step - accuracy: 0.6145 - loss: 1.2770
Epoch 12

In [ ]:
# Generate next character predictions
seed_text = "There are several kin"
next_chars = 20

for _ in range(next_chars):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_char = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += predicted_char

print("Next predicted characters:", seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Next predicted characters: There are several kinetion of the problem
